In [1]:
import tensorflow as tf

2024-03-23 00:46:22.204096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 00:46:22.204224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 00:46:22.336797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Modified MIG

In [2]:
@tf.function
def MIG(model, images, labels, size, iterations = 20, steps = 15):
    # interpolated intensities
    alphas = tf.linspace(0.0,1.0,steps+1)
    alphas_x = alphas[:,tf.newaxis, tf.newaxis, tf.newaxis,tf.newaxis]
    alphas_x = tf.reshape(alphas_x,(1,alphas.shape[0],1,1,1))
    images = tf.cast(images,'float32')
    g_t_0 = tf.zeros_like(images)
    g_t_1 = tf.zeros_like(images)
    g_t_2 = tf.zeros_like(images)
    g_t_3 = tf.zeros_like(images)
    g_t_4 = tf.zeros_like(images)
    x_t_0 = images
    x_t_1 = images
    x_t_2 = images
    x_t_3 = images
    x_t_4 = images
    #eps_step = tf.constant(epsilon/iterations)

    for i in tf.range(iterations):
        # calculate interpolated images
        delta = tf.expand_dims(x_t_4,1)
        interpolated = delta * alphas_x
        # calculate gradient for each image
        grads = tf.TensorArray(tf.float32,size)
        i = 0
        for inter in interpolated:
            with tf.GradientTape() as tape:
                tape.watch(inter)
                probs = model(inter)[:,labels[i]]
            grads = grads.write(i,tape.gradient(probs,inter))
            i+= 1

        grads = grads.stack()
        # aproximate integration using remain trapoziod
        grads = (grads[:,:-1] + grads[:,1:]) / tf.constant(2.0)
        avg_gradients = tf.math.reduce_mean(grads, axis=1)

        integrated_gradient = x_t_4 * avg_gradients
        #delta_t = self.IG(x_t,labels,size)

        g_t_4 = g_t_4 + (integrated_gradient/tf.norm(integrated_gradient,1))
        x_h = x_t_4 - 0.25 * tf.sign(g_t_4)
        x_t_4 = tf.clip_by_value(x_h,x_h-5,x_h+5)

        #integrated_gradient = tf.clip_by_value(integrated_gradient,integrated_gradient-4,integrated_gradient+4)
        g_t_3 = g_t_3 + (integrated_gradient/tf.norm(integrated_gradient,1))
        x_h = x_t_3 - 0.2 * tf.sign(g_t_3)
        x_t_3 = tf.clip_by_value(x_h,x_h-4,x_h+4)

        #integrated_gradient = tf.clip_by_value(integrated_gradient,integrated_gradient-3,integrated_gradient+3)
        g_t_2 = g_t_2 + (integrated_gradient/tf.norm(integrated_gradient,1))
        x_h = x_t_2 - 0.15 * tf.sign(g_t_2)
        x_t_2 = tf.clip_by_value(x_h,x_h-3,x_h+3)

        #integrated_gradient = tf.clip_by_value(integrated_gradient,integrated_gradient-2,integrated_gradient+2)
        g_t_1 = g_t_1 + (integrated_gradient/tf.norm(integrated_gradient,1))
        x_h = x_t_1 - 0.1 * tf.sign(g_t_1)
        x_t_1 = tf.clip_by_value(x_h,x_h-2,x_h+2)

        #integrated_gradient = tf.clip_by_value(integrated_gradient,integrated_gradient-1,integrated_gradient+1)
        g_t_0 = g_t_0 + (integrated_gradient/tf.norm(integrated_gradient,1))
        x_h = x_t_0 - 0.05 * tf.sign(g_t_1)
        x_t_0 = tf.clip_by_value(x_h,x_h-1,x_h+1)

    return tf.stack([tf.clip_by_value(x_t_0,0,255),tf.clip_by_value(x_t_1,0,255),tf.clip_by_value(x_t_2,0,255),tf.clip_by_value(x_t_3,0,255),tf.clip_by_value(x_t_4,0,255)])

In [3]:
eval_dir = "/kaggle/input/evaluation-of-imagnet/evaluation"
eval_set = tf.keras.utils.image_dataset_from_directory(eval_dir,image_size=(224, 224),batch_size=4, seed = 4854)

Found 50000 files belonging to 1000 classes.


# Create subset of evaluation

In [5]:
clean_set = []
label_set = []

extracted_label = []
extracted_clean = []

eps = [1,2,3,4,5]
eps_iter = [e/20 for e in eps]

cnt = 0

for batch, label in eval_set:
    clean_set.append(batch)
    label_set.append(label)
    for i in label:
        extracted_label.append(i)
    for i in batch:
        extracted_clean.append(i)
    cnt += 1
    if cnt == 400:
        break


extracted_label = tf.convert_to_tensor(extracted_label)
extracted_clean = tf.convert_to_tensor(extracted_clean)

In [9]:
metrics_new = [tf.keras.metrics.SparseCategoricalCrossentropy(name="migloss1"),
        tf.keras.metrics.SparseCategoricalCrossentropy(name="migloss2"),
        tf.keras.metrics.SparseCategoricalCrossentropy(name="migloss3"),
        tf.keras.metrics.SparseCategoricalCrossentropy(name="migloss4"),
        tf.keras.metrics.SparseCategoricalCrossentropy(name="migloss5"),
        tf.keras.metrics.SparseCategoricalAccuracy(name='migacc1'),
        tf.keras.metrics.SparseCategoricalAccuracy(name='migacc2'),
        tf.keras.metrics.SparseCategoricalAccuracy(name='migacc3'),
        tf.keras.metrics.SparseCategoricalAccuracy(name='migacc4'),
        tf.keras.metrics.SparseCategoricalAccuracy(name='migacc5')]

In [10]:
base_model=tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    classifier_activation='softmax',
    classes=1000
)

inputs = tf.keras.layers.Input((224, 224, 3))
preprocessing = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input)(inputs)
outputs = base_model(preprocessing)

base_model= tf.keras.Model(inputs = inputs, outputs = outputs)
base_model.compile(optimizer="sgd",loss = "sparse_categorical_crossentropy",metrics=['sparse_categorical_accuracy'])
base_model.load_weights('/kaggle/input/efficientnetb0-adv-pgd/checkpoint.h5')

In [11]:
results_new = base_model.evaluate(extracted_clean, extracted_label)
        
#Evaluation newmodel
for batch, label in zip(clean_set, label_set):
    adv = MIG(base_model, batch, label, 4)
    for i in range(5):
        y_pred = base_model(adv[i], training=False)
        metrics_new[i].update_state(label, y_pred)
        metrics_new[i+5].update_state(label, y_pred)

50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - loss: 1.4425 - sparse_categorical_accuracy: 0.6572


In [12]:
with open('Evaluate_newmodel.txt','w') as file:
    file.write(f'cleanacc: {results_new[1]:.4f}\n')
    file.write(f'cleanloss: {results_new[0]:.4f}\n')
    for mb in metrics_new:
        file.write(f"{mb.name}: {mb.result():.4f}\n")

# Old method Evaluation

In [14]:
@tf.function
def IG(model, images, labels, size, steps = 15):
    # interpolated intensities
    alphas = tf.linspace(0.0,1.0,steps+1)
    alphas_x = alphas[:,tf.newaxis, tf.newaxis, tf.newaxis,tf.newaxis]
    alphas_x = tf.reshape(alphas_x,(1,alphas.shape[0],1,1,1))

    # standardize input
    images = tf.cast(images,'float32')

    # calculate interpolated images
    delta = tf.expand_dims(images,1)
    interpolated = delta * alphas_x

    # calculate gradient for each image
    grads = tf.TensorArray(tf.float32,32)
    i = 0
    for inter in interpolated:
        with tf.GradientTape() as tape:
            tape.watch(inter)
            probs = model(inter)[:,labels[i]]
        grads = grads.write(i,tape.gradient(probs,inter))
        i+= 1

    grads = grads.stack()

    # aproximate integration using remain trapoziod
    grads = (grads[:,:-1] + grads[:,1:]) / tf.constant(2.0)
    avg_gradients = tf.math.reduce_mean(grads, axis=1)

    integrated_gradient = images * avg_gradients

    return integrated_gradient

@tf.function
def MIG(model, images, labels, size, epsilon = 16, iterations = 20):
    g_t = tf.zeros_like(images)
    x_t = images
    eps_step = tf.constant(epsilon/iterations)
    for i in tf.range(iterations):
        # steps = 20 as research
        #labels = tf.argmax(model(x_t) , -1)
        delta_t = IG(model,x_t,labels,size) 
        g_t = g_t + (delta_t/tf.norm(delta_t,1))
        x_h = x_t - eps_step * tf.sign(g_t)
        x_t = tf.clip_by_value(x_h,x_h-epsilon,x_h+epsilon)
    return tf.clip_by_value(x_t,0,255)